In [1]:
from keras.models import Sequential

# The following contain functions that are specific to Convolutional Networks.
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout

# RMSprop is a type of optimizer.
from keras.optimizers import RMSprop

from keras.datasets import mnist
from keras.utils import np_utils

from keras import initializers

# Backend can either be 'tensorflow' or 'theano'.
from keras import backend as K

Using TensorFlow backend.
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/adgdri/anaconda3

In [2]:
# Model Parameters.
batch_size = 128
nb_classes = 10
nb_epoch = 100

In [3]:
img_rows, img_cols = 28, 28         # input image dimensions
pool_size = (2, 2)                  # size of pooling area for max pooling
prob_drop_conv = 0.2                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

In [5]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

# # The following needs to be done, because the order of dimensions are different in theano and tensorflow.
# if K.image_dim_ordering() == 'th':
#     # For Theano backend
#     X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#     X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     # For TensorFlow backend
#     X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
#     X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

X_train original shape: (60000, 28, 28)


In [8]:
# For TensorFlow backend
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [9]:
# Normalizing the input data.
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Conversion to one-hot encoding of the class vectors.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# Building the Convolutional Model
model = Sequential()

# Conv1 layer
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu', input_shape=input_shape, init=initializers.normal()))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# Conv2 layer
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu', init=initializers.normal()))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# Conv3 layer
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu', init=initializers.normal()))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# While building any convolutional networks, you should add a couple of dense layers at the end of the network.
# FC1 layer
model.add(Dense(625, activation='relu', init=initializers.normal()))
model.add(Dropout(prob_drop_hidden))

# The last layer of the classifier mostly has a 'softmax' activation unit, to get the probabilities.
# FC2 layer
model.add(Dense(10, activation='softmax', init=initializers.normal()))

# Declaring a separate object for the optimizer.
opt = RMSprop(lr=0.001, rho=0.9)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1..., padding="same", kernel_initializer=<keras.ini...)`
  """
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same", kernel_initializer=<keras.ini...)`
  # Remove the CWD from sys.path while we load stuff.
/Users/adgdri/anaconda3/envs/work/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 128)         73856     
__________

In [7]:
# Train
history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1)

Epoch 1/100
60000/60000 [==============================] - 6s - loss: 0.4027 - acc: 0.8651     
Epoch 2/100
60000/60000 [==============================] - 5s - loss: 0.1045 - acc: 0.9677     
Epoch 3/100
60000/60000 [==============================] - 5s - loss: 0.0732 - acc: 0.9773     
Epoch 4/100
60000/60000 [==============================] - 5s - loss: 0.0572 - acc: 0.9819     
Epoch 5/100
60000/60000 [==============================] - 5s - loss: 0.0493 - acc: 0.9848     
Epoch 6/100
60000/60000 [==============================] - 5s - loss: 0.0444 - acc: 0.9867     
Epoch 7/100
60000/60000 [==============================] - 5s - loss: 0.0407 - acc: 0.9878     
Epoch 8/100
60000/60000 [==============================] - 5s - loss: 0.0379 - acc: 0.9882     
Epoch 9/100
60000/60000 [==============================] - 5s - loss: 0.0354 - acc: 0.9897     
Epoch 10/100
60000/60000 [==============================] - 5s - loss: 0.0341 - acc: 0.9897     
Epoch 11/100
60000/60000 [=============

In [8]:
# Evaluate
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

 9984/10000 [============================>.] - ETA: 0sSummary: Loss over the test dataset: 0.03, Accuracy: 0.99
